In [1]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [2]:
# LTL Tariff - no distance
def get_tariff(weight):
    tariff_levels = pd.Series({1.79:0,
                    1.70:201,
                    1.62:501,
                    1.53:1001,
                    1.46:1501,
                    1.38:2001,
                    1.32:3001,
                    1.25:4001,
                    1.19:5001,
                    1.13:7501,
                    1.07:10001})
    cost=tariff_levels[tariff_levels<=weight].index[-1]
    
    tariff = cost*weight
    
    if tariff>=250:
        return tariff
    else: 
        return 250
    

In [3]:
get_tariff(800)

1296.0

In [4]:
# LTL Tariff - with distance
def get_tariff_dist(weight,dist):
    tariff_levels = pd.read_csv("./Data/LTLTariff.csv",index_col=0)
    tariff_levels.columns = tariff_levels.columns.astype(float)
    
    x=tariff_levels.columns[tariff_levels.columns <= dist][-1]
    y=tariff_levels.index[tariff_levels.index <= weight][-1]
    
    tariff = tariff_levels.loc[y,x]   
    
    
    return tariff

In [5]:
get_tariff_dist(800,50)

57.56

In [6]:
# FTL Tariff
def get_tariff_ftl(dist):
    transport_cost = 50
    transport_time = 0 #will need to be updated if this comes into play
    distance_rate = 1.4 # Euro/km
    return transport_cost+(dist*distance_rate)

In [7]:
get_tariff_ftl(250)

400.0

In [8]:
# Milk Run Tariff
def get_tariff_milk(dist, num_stops):
    transport_cost = 100
    transport_time = 0 #will need to be updated if this comes into play
    distance_rate = 0.6
    stop_cost = 40
    return transport_cost + distance_rate*dist + num_stops*stop_cost

In [9]:
get_tariff_milk(250, 2)

330.0

In [10]:
# Means of Transportation Class
class MoT():
    def __init__(self,name,avg_speed,payload,length,width,height):
        self.name = name
        self.avg_speed = avg_speed
        self.max_payload = payload
        self.max_length = length
        self.max_width = width
        self.max_height = height
        self.rem_length = length
        self.rem_width = width
        self.rem_height = height
        self.rem_payload = payload
        self.max_vol = length*width*height
        self.TOs_loaded = []
    
    def load_truck(self,TO):
        if self._check_fits()
    
    def unload_truck(self,TO):
        pass
    
    def empty_truck(self,TO):
        pass
    
    def _check_fits(self,TO):
        fits = True
        if self.rem_width < TO.

SyntaxError: invalid syntax (<ipython-input-10-b6fabc03f5db>, line 18)

In [11]:
a = MoT('Standard 25to',50,25,13600,2500,2480)
b = MoT('MEGA',70,25,13620,2480,3000)
c = MoT('PICKUP 3.5t',60,3.5,6400,2500,2500)

NameError: name 'MoT' is not defined

In [12]:
# Transport Order Class
class Order():
    def __init__(self,order_num,origin,destination,weight,length,volume):
        self.order_num = order_num
        self.origin = origin
        self.destination = destination
        self.weight = weight
        self.length = length
        self.volume = volume
   
    

In [13]:
class Tour():
    #add stop
    #

SyntaxError: unexpected EOF while parsing (<ipython-input-13-94cb91355259>, line 3)

In [74]:
TOs = pd.read_csv("./Data/TransportOrders.csv")

In [75]:
TOs['Weight'] = pd.to_numeric(TOs['Weight'].str.rstrip(' kg').str.replace(',',''))
TOs['Loading Meters'] = pd.to_numeric(TOs['Loading Meters'].str.split(expand=True)[0])
TOs['Volume'] = pd.to_numeric(TOs['Volume'].str.split(expand=True)[0])

In [76]:
TO_list = []
for index, row in TOs.iterrows():
    TO_list.append(Order(row['Transport Order'], 
                         row['Origin Index'], 
                         row['Destination Index'], 
                         row['Weight'], 
                         row['Loading Meters'], 
                         row['Volume'])
                  )
    
    

In [77]:
data = {}
#Define Distance Matrix
data['distance_matrix'] = [
    [0,170,210,2219,1444],
    [170,0,243,2253,1369],
    [210,243,0,2042,1267],
    [2219,2253,2042,0,1127],
    [1444,1369,1267,1127,0]
]

data['pickups_deliveries'] = []
for i in TO_list:
    data['pickups_deliveries'].append([i.origin,i.destination])

data['num_vehicles'] = 50
data['depot']=0


In [78]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Model

In [79]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))



In [80]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])


In [81]:
routing = pywrapcp.RoutingModel(manager)

In [82]:
def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [83]:
dimension_name = 'Distance'
routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

In [84]:
for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))


In [85]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)


In [86]:
solution = routing.SolveWithParameters(search_parameters)


In [87]:
if solution:
        print_solution(data, manager, routing, solution)